In [ ]:
# NFL Draft Prediction - 探索的データ分析（EDA）

## 概要
NFL選手のドラフト指名予測のための包括的なデータ分析を行います。

## 目標
- データの構造と品質を理解
- 欠損値パターンの分析
- 特徴量とターゲットの関係性の発見
- ドメイン知識の活用とインサイトの獲得


In [ ]:
## 1. セットアップ


In [ ]:
# 基本ライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats

# 設定
plt.style.use('default')
sns.set_palette("husl")
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# 日本語フォント設定（必要に応じて）
plt.rcParams['font.family'] = ['DejaVu Sans', 'Arial', 'sans-serif']

print("ライブラリの読み込み完了")


In [ ]:
## 2. データ読み込み


In [ ]:
# データ読み込み
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sample_submission = pd.read_csv('data/sample_submission_6_20修正.csv')

print(f"訓練データサイズ: {train.shape}")
print(f"テストデータサイズ: {test.shape}")
print(f"サンプル提出ファイルサイズ: {sample_submission.shape}")

# データの最初の数行を確認
print("\n=== 訓練データ（先頭5行）===")
display(train.head())
print("\n=== テストデータ（先頭5行）===")
display(test.head())


In [ ]:
## 3. 基本統計量とデータ概要


In [ ]:
# データの基本情報
print("=== 訓練データの基本情報 ===")
print(train.info())
print("\n=== 訓練データの基本統計量 ===")
print(train.describe())

# カテゴリ変数の確認
print("\n=== カテゴリ変数の概要 ===")
categorical_cols = train.select_dtypes(include=['object']).columns.tolist()
for col in categorical_cols:
    print(f"\n{col}: {train[col].nunique()}個のユニーク値")
    print(train[col].value_counts().head())


In [ ]:
## 4. 目的変数（Drafted）の分析


In [ ]:
# 目的変数の分布分析
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# ヒストグラム
axes[0].hist(train['Drafted'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
axes[0].set_title('Drafted値の分布')
axes[0].set_xlabel('Drafted')
axes[0].set_ylabel('頻度')

# 二値化での円グラフ
drafted_binary = (train['Drafted'] > 0.5).astype(int)
drafted_counts = drafted_binary.value_counts()
axes[1].pie(drafted_counts.values, labels=['指名されず', '指名された'], autopct='%1.1f%%',
           colors=['lightcoral', 'lightgreen'])
axes[1].set_title('ドラフト指名の割合（二値化）')

plt.tight_layout()
plt.show()

print(f"Drafted値の平均: {train['Drafted'].mean():.4f}")
print(f"指名された選手の割合: {drafted_binary.mean():.4f}")
print(f"指名された選手数: {drafted_binary.sum()}人")
print(f"指名されなかった選手数: {len(drafted_binary) - drafted_binary.sum()}人")


In [ ]:
## 5. 欠損値の詳細分析


In [ ]:
# 欠損値の詳細分析
def analyze_missing_values(df, title):
    missing_data = df.isnull().sum()
    missing_percent = 100 * missing_data / len(df)
    
    missing_table = pd.DataFrame({
        '欠損数': missing_data,
        '欠損率(%)': missing_percent
    })
    missing_table = missing_table[missing_table['欠損数'] > 0].sort_values('欠損数', ascending=False)
    
    print(f"=== {title} 欠損値分析 ===")
    if len(missing_table) > 0:
        print(missing_table)
        
        # 可視化
        plt.figure(figsize=(10, 6))
        sns.barplot(data=missing_table.reset_index(), x='欠損率(%)', y='index')
        plt.title(f'{title} - 欠損率(%)')
        plt.tight_layout()
        plt.show()
    else:
        print("欠損値はありません")
    
    return missing_table

# 訓練データとテストデータの欠損値分析
train_missing = analyze_missing_values(train, "訓練データ")
test_missing = analyze_missing_values(test, "テストデータ")


In [ ]:
## 6. ポジション別の指名傾向分析


In [ ]:
# ポジション別の指名率分析
def analyze_position_draft_rate(df):
    position_columns = ['Player_Type', 'Position_Type', 'Position']
    
    for pos_col in position_columns:
        # 統計量計算
        stats = df.groupby(pos_col).agg({
            'Drafted': ['count', 'mean', 'std']
        }).round(3)
        stats.columns = ['総選手数', '指名率', '指名率_std']
        stats = stats.sort_values('指名率', ascending=False)
        
        print(f"\n=== {pos_col}別の指名率 ===")
        print(stats.head(10))
        
        # 可視化（上位15ポジション）
        plt.figure(figsize=(12, 8))
        top_positions = stats.head(15)
        
        # 棒グラフ作成
        bars = plt.barh(range(len(top_positions)), top_positions['指名率'])
        plt.yticks(range(len(top_positions)), top_positions.index)
        plt.xlabel('指名率')
        plt.title(f'{pos_col}別指名率（上位15）')
        
        # 選手数を棒グラフに表示
        for i, (idx, row) in enumerate(top_positions.iterrows()):
            plt.text(row['指名率'] + 0.01, i, f"n={int(row['総選手数'])}", 
                    va='center', fontsize=9)
        
        plt.tight_layout()
        plt.show()

analyze_position_draft_rate(train)


In [ ]:
## 7. 学校別の指名傾向分析


In [ ]:
# 学校別の分析（複数選手を持つ学校のみ）
school_stats = train.groupby('School').agg({
    'Drafted': ['count', 'mean']
}).round(3)
school_stats.columns = ['選手数', '指名率']

# 5人以上の選手を持つ学校のみを分析対象とする
significant_schools = school_stats[school_stats['選手数'] >= 5].copy()
significant_schools = significant_schools.sort_values('指名率', ascending=False)

print("=== 学校別指名率（5人以上の学校、上位20）===")
print(significant_schools.head(20))

# 可視化
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# 1. 散布図：選手数 vs 指名率
axes[0].scatter(significant_schools['選手数'], significant_schools['指名率'], 
               s=60, alpha=0.7, c='skyblue', edgecolors='black')

# 上位10校にラベル追加
top_10_schools = significant_schools.head(10)
for school, row in top_10_schools.iterrows():
    axes[0].annotate(school, (row['選手数'], row['指名率']), 
                    xytext=(5, 5), textcoords='offset points', 
                    fontsize=8, ha='left')

axes[0].set_xlabel('選手数')
axes[0].set_ylabel('指名率')
axes[0].set_title('学校別：選手数と指名率の関係')
axes[0].grid(True, alpha=0.3)

# 2. 上位15校の指名率
top_15_schools = significant_schools.head(15)
bars = axes[1].barh(range(len(top_15_schools)), top_15_schools['指名率'])
axes[1].set_yticks(range(len(top_15_schools)))
axes[1].set_yticklabels(top_15_schools.index)
axes[1].set_xlabel('指名率')
axes[1].set_title('学校別指名率（上位15校）')

# 選手数を表示
for i, (school, row) in enumerate(top_15_schools.iterrows()):
    axes[1].text(row['指名率'] + 0.01, i, f"n={int(row['選手数'])}", 
                va='center', fontsize=9)

plt.tight_layout()
plt.show()

# 学校プレステージの分析
print(f"\n=== 学校カテゴリ分析 ===")
print(f"分析対象学校数: {len(significant_schools)}")
print(f"高指名率校（0.8以上）: {len(significant_schools[significant_schools['指名率'] >= 0.8])}校")
print(f"中指名率校（0.4-0.8）: {len(significant_schools[(significant_schools['指名率'] >= 0.4) & (significant_schools['指名率'] < 0.8)])}校")
print(f"低指名率校（0.4未満）: {len(significant_schools[significant_schools['指名率'] < 0.4])}校")


In [ ]:
## 8. 年度別のトレンド分析


In [ ]:
# 年度別の指名率と選手数の分析
year_stats = train.groupby('Year').agg({
    'Drafted': ['count', 'mean', 'std'],
    'Height': 'mean',
    'Weight': 'mean',
    'Sprint_40yd': 'mean'
}).round(3)

# 列名を整理
year_stats.columns = ['選手数', '指名率', '指名率_std', '平均身長', '平均体重', '平均40ヤード走']

print("=== 年度別統計 ===")
print(year_stats)

# 可視化
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. 選手数の推移
axes[0, 0].plot(year_stats.index, year_stats['選手数'], marker='o', linewidth=2, markersize=8)
axes[0, 0].set_title('年度別選手数の推移')
axes[0, 0].set_xlabel('年度')
axes[0, 0].set_ylabel('選手数')
axes[0, 0].grid(True, alpha=0.3)

# 2. 指名率の推移（エラーバー付き）
axes[0, 1].errorbar(year_stats.index, year_stats['指名率'], 
                   yerr=year_stats['指名率_std'], 
                   marker='o', linewidth=2, markersize=8, capsize=5)
axes[0, 1].set_title('年度別指名率の推移')
axes[0, 1].set_xlabel('年度')
axes[0, 1].set_ylabel('指名率')
axes[0, 1].grid(True, alpha=0.3)

# 3. 平均身長の推移
axes[1, 0].plot(year_stats.index, year_stats['平均身長'], marker='s', linewidth=2, markersize=8, color='green')
axes[1, 0].set_title('年度別平均身長の推移')
axes[1, 0].set_xlabel('年度')
axes[1, 0].set_ylabel('平均身長 (m)')
axes[1, 0].grid(True, alpha=0.3)

# 4. 平均40ヤード走の推移
axes[1, 1].plot(year_stats.index, year_stats['平均40ヤード走'], marker='^', linewidth=2, markersize=8, color='red')
axes[1, 1].set_title('年度別平均40ヤード走の推移')
axes[1, 1].set_xlabel('年度')
axes[1, 1].set_ylabel('平均40ヤード走 (秒)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 年度別のポジション分布変化
print(f"\n=== 年度別ポジションタイプ分布 ===")
position_year_crosstab = pd.crosstab(train['Year'], train['Position_Type'], normalize='index')
print(position_year_crosstab.round(3))

# ヒートマップで可視化
plt.figure(figsize=(12, 8))
sns.heatmap(position_year_crosstab.T, annot=True, cmap='Blues', fmt='.2f')
plt.title('年度別ポジションタイプ分布（割合）')
plt.xlabel('年度')
plt.ylabel('ポジションタイプ')
plt.tight_layout()
plt.show()


In [ ]:
## 9. 身体能力指標と指名確率の関係性分析


In [ ]:
# 身体能力指標と指名確率の詳細分析
performance_cols = ['Sprint_40yd', 'Vertical_Jump', 'Bench_Press_Reps', 'Broad_Jump', 
                   'Agility_3cone', 'Shuttle', 'Height', 'Weight', 'Age']

# 指名された選手 vs されなかった選手の統計比較
drafted_players = train[train['Drafted'] > 0.5]
undrafted_players = train[train['Drafted'] <= 0.5]

comparison_stats = pd.DataFrame({
    '指名_平均': drafted_players[performance_cols].mean(),
    '非指名_平均': undrafted_players[performance_cols].mean(),
    '指名_中央値': drafted_players[performance_cols].median(),
    '非指名_中央値': undrafted_players[performance_cols].median(),
    '指名_標準偏差': drafted_players[performance_cols].std(),
    '非指名_標準偏差': undrafted_players[performance_cols].std()
})

comparison_stats['平均差'] = comparison_stats['指名_平均'] - comparison_stats['非指名_平均']
comparison_stats['中央値差'] = comparison_stats['指名_中央値'] - comparison_stats['非指名_中央値']
comparison_stats['効果サイズ'] = comparison_stats['平均差'] / ((comparison_stats['指名_標準偏差'] + comparison_stats['非指名_標準偏差']) / 2)

print("=== 指名状況別の身体能力比較 ===")
print(comparison_stats.round(3))

# 効果サイズによる重要度ランキング
effect_size_ranking = comparison_stats['効果サイズ'].abs().sort_values(ascending=False)
print(f"\n=== 効果サイズランキング（絶対値） ===")
print(effect_size_ranking)


In [ ]:
# 身体能力指標のボックスプロット比較
n_cols = 3
n_rows = (len(performance_cols) + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4*n_rows))
axes = axes.flatten()

# Draftedを二値化
train_viz = train.copy()
train_viz['Draft_Status'] = train_viz['Drafted'].apply(lambda x: '指名' if x > 0.5 else '非指名')

for i, col in enumerate(performance_cols):
    if i < len(axes):
        sns.boxplot(data=train_viz, x='Draft_Status', y=col, ax=axes[i])
        axes[i].set_title(f'{col}の分布（指名状況別）')
        axes[i].tick_params(axis='x', rotation=0)

# 余ったサブプロットを非表示
for i in range(len(performance_cols), len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.show()

# 身体能力指標間の相関分析
performance_data = train[performance_cols + ['Drafted']].dropna()
correlation_matrix = performance_data.corr()

plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
           square=True, linewidths=0.5, fmt='.2f')
plt.title('身体能力指標の相関マトリックス')
plt.tight_layout()
plt.show()

# Draftedとの相関ランキング
drafted_corr = correlation_matrix['Drafted'].abs().sort_values(ascending=False)
print(f"\n=== Draftedとの相関ランキング ===")
print(drafted_corr.drop('Drafted').head(10))
